In [1]:
# 1. Import Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tweepy
from tweepy import OAuthHandler

In [2]:
# 2.Set Connecion

#Get from developers.twitter.com/App->Setting->keys&tokens
#Just assign the credentials

consumer_key = "xhsSu7y92oAFi9U3yX794Chav"
consumer_secret = "0YWxdhZ2XvBs6Tk33iolwIYSdcuZL23CmhRK1TeHV6bIMdQ7SM"
access_token = "324351622-rO9AGKtQ8WvWBQrX8bnWlEvXcJ3FgKpR3tPLB7fW"
access_token_secret = "8OAe9FSFELuvIRHccwOgncxAgxIPgGvp19JDe44IeYOuV"

# Use the above credentials to authenticate the API.

auth = tweepy.OAuthHandler( consumer_key , consumer_secret )
auth.set_access_token( access_token , access_token_secret )
api = tweepy.API(auth)

In [3]:
import json
import pymongo

In [4]:
# 3.To get the tweets in a Proper format, first lets create a Dataframe to store the extracted data.

df = pd.DataFrame(columns=["Date","User","IsVerified","Tweet","Likes","RT",'User_location'])
print(df)

Empty DataFrame
Columns: [Date, User, IsVerified, Tweet, Likes, RT, User_location]
Index: []


In [ ]:
connection = MongoClient('localhost', 27017)
db = connection.test_database
db.test.ensure_index("id", unique=True, dropDups=True)
collection = db.test

In [12]:
# We will use api as api.search inside this tweepy cursor.

# We will Use **tweepy.cursor()** because we want to extract a larger number of tweets i.e over 100,500 etc

def get_tweets(Topic,Count):    
    i=0
    for tweet in tweepy.Cursor(api.search, q=Topic,count=100, lang="en",exclude='retweets').items():
        print(i, end='\r')
        df.loc[i,"Date"] = tweet.created_at
        df.loc[i,"User"] = tweet.user.name
        df.loc[i,"IsVerified"] = tweet.user.verified
        df.loc[i,"Tweet"] = tweet.text
        df.loc[i,"Likes"] = tweet.favorite_count
        df.loc[i,"RT"] = tweet.retweet_count
        df.loc[i,"User_location"] = tweet.user.location
#         df.to_csv("TweetDataset.csv",index=False)
#         df.to_excel('{}.xlsx'.format("TweetDataset"),index=False)   ## Save as Excel
        parsed = json.loads(df.to_json(orient="records"))
        print(parsed)
#         json.dumps(parsed, indent=4)
#         print(json.dumps(parsed, indent=4))
        i=i+1
        if i>Count:
            break
        else:
            pass

In [13]:
# Call the function to extract the data. pass the topic and filename you want the data to be stored in.
Topic=["Starbucks"]
get_tweets(Topic , Count=100)

[{'Date': 1617810547000, 'User': 'Fcuk Derek Chauvin', 'IsVerified': False, 'Tweet': '@1983Mo @Starbucks Not that I’ve seen.', 'Likes': 0, 'RT': 0, 'User_location': 'Maryland, USA'}, {'Date': 1617810542000, 'User': 'Chan✨', 'IsVerified': False, 'Tweet': 'I’m sorry but a starbucks drive thru should take 5 minutes tops. Only drinks are being made.', 'Likes': 0, 'RT': 0, 'User_location': 'Houston, TX'}, {'Date': 1617810541000, 'User': 'MC Connors', 'IsVerified': False, 'Tweet': '@NoraPrinciotti Is “get up each day/get a beach day” the new “starbucks lovers??” Need to know.', 'Likes': 0, 'RT': 0, 'User_location': ''}, {'Date': 1617810519000, 'User': '𝚜𝚠𝚊𝚖𝚙 𝚍𝚎𝚖𝚘𝚗', 'IsVerified': False, 'Tweet': 'Pro tip: if you’re addicted like me and Starbucks is out of the brown sugar syrup, you can sub cinnamon dolce and i… https://t.co/OXENvSzwY9', 'Likes': 0, 'RT': 0, 'User_location': 'NE•TX•MN•AR•CO'}, {'Date': 1617810513000, 'User': 'Kate', 'IsVerified': False, 'Tweet': 'I got a kiwi star fruit refre

In [ ]:
from pymongo import MongoClient
import json
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import datetime


# The MongoDB connection info. This assumes your database name is twitter_database, and your collection name is tweets.
connection = MongoClient('localhost', 27017)
db = connection.twitter_database
db.tweets.ensure_index("id", unique=True, dropDups=True)
collection = db.tweets

# Add the keywords you want to track. They can be cashtags, hashtags, or words.
keywords = ['starbucks', '#starbucks']

# Optional - Only grab tweets of specific language
language = ['en']

# You need to replace these with your own values that you get after creating an app on Twitter's developer portal.
consumer_key = "xhsSu7y92oAFi9U3yX794Chav"
consumer_secret = "0YWxdhZ2XvBs6Tk33iolwIYSdcuZL23CmhRK1TeHV6bIMdQ7SM"
access_token = "324351622-rO9AGKtQ8WvWBQrX8bnWlEvXcJ3FgKpR3tPLB7fW"
access_token_secret = "8OAe9FSFELuvIRHccwOgncxAgxIPgGvp19JDe44IeYOuV"

# The below code will get Tweets from the stream and store only the important fields to your database
class StdOutListener(StreamListener):

    def on_data(self, data):

        # Load the Tweet into the variable "t"
        t = json.loads(data)

        # Pull important data from the tweet to store in the database.
        tweet_id = t['id_str']  # The Tweet ID from Twitter in string format
        created_at = t['created_at']
        username = t['user']['screen_name']  # The username of the Tweet author
        followers = t['user']['followers_count']  # The number of followers the Tweet author has
        text = t['text']  # The entire body of the Tweet
        favorite_count = t['favorite_count']
        hashtags = t['entities']['hashtags']  # Any hashtags used in the Tweet
        retweet_count = t['retweet_count']
        location = t['user']['location']
        dt = t['created_at']  # The timestamp of when the Tweet was created
        language = t['lang']  # The language of the Tweet

        # Convert the timestamp string given by Twitter to a date object called "created". This is more easily manipulated in MongoDB.
        created = datetime.datetime.strptime(dt, '%a %b %d %H:%M:%S +0000 %Y')

        # Load all of the extracted Tweet data into the variable "tweet" that will be stored into the database
        tweet = {'id':tweet_id, 'created_at':created_at, 'username':username, 'followers':followers, 'tweet':text, 'favorite_count':favorite_count,'hashtags':hashtags, 'retweet_count':retweet_count, 'location':location, 'language':language, 'created':created}

        # Save the refined Tweet data to MongoDB
        collection.save(tweet)

        # Optional - Print the username and text of each Tweet to your console in realtime as they are pulled from the stream
        print(username + ':' + ' ' + text)
        return True
        
    # Prints the reason for an error to your console
    def on_error(self, status):
        print(status)

# Some Tweepy code that can be left alone. It pulls from variables at the top of the script
if __name__ == '__main__':
    l = StdOutListener()
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)

    stream = Stream(auth, l)
    stream.filter(track=keywords, languages=language)

In [36]:
import sys
import pymongo
from pymongo import MongoClient
connection = MongoClient('localhost', 27017)
db = connection.twitter_database
collection = db.tweets_data

import pandas as pd
# from pandas import DataFrame
data = db.collection.find({'Likes':0})
tweet_fields = ['Likes']
result = DataFrame(list(data), columns = tweet_fields)
# print(result)
# result.tail()
# print(data)
# df = pd.DataFrame.from_records(data)
# print(df)

# client = pymongo.MongoClient()
# data = db.mydb.mycollection.find() # or db.mydb.mycollection.aggregate(pipeline)

# df = pd.DataFrame.from_records(data)

,Likes


In [ ]:
def get_tweets(Topic,Count):    
    i=0
    for tweet in tweepy.Cursor(api.search, q=Topic,count=100, lang="en",exclude='retweets').items():
        metadata = tweet.user.result_type
        print(metadata)
        i=i+1
        if i>Count:
            break
        else:
            pass

Topic=["Starbucks"]
get_tweets(Topic , Count=1)